In [1]:
import pandas as pd
from nltk import word_tokenize
# Remoção de acentos
from unicodedata import normalize
import _thread
import time
import requests
import re


# Tokenização com nltk
from nltk import word_tokenize

In [2]:
df = pd.read_csv('noticias_if.csv')
df = df.iloc[10:]

In [3]:
df = df[(df['descricao'].isnull() == False)]

In [4]:
import nltk

stopwords = nltk.corpus.stopwords.words('portuguese')
#print(stopwords)

In [5]:
from string import punctuation

punctuation+'“”–'

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~“”–'

In [6]:
def processa(row):
    txt = row['titulo'] + ' ' + row['descricao']
    
    txt = re.sub('<[^<]+?>', '', txt)
    
    return ' '.join([t for t in word_tokenize(txt.lower()) if (t not in stopwords) and (t not in punctuation)])

df['doc'] = df.apply(processa, axis=1)

df['doc'].head()

11    fapemig lança pacote editais 2014 fundação amp...
13    conselho pais realiza “ dia campo ” pais estud...
14    censoead.br 2012 2013 censoead.br 2012 objetiv...
17    integrantes seção cultura artes participam eve...
19    período inscrições sisu 2014 6 10 janeiro if s...
Name: doc, dtype: object

In [7]:
df.iloc[0].doc

'fapemig lança pacote editais 2014 fundação amparo pesquisa estado minas gerais fapemig lançou pacote editais ano 2014 destinando r 36 milhões recursos apoio pesquisadores mineiros lançamentos edital demanda universal provê apoio projetos qualquer área conhecimento demais modalidades abertas oferecem suporte trabalhos pesquisa desenvolvidos estado meio concessão bolsas pesquisadores financiamento itens apoio livros manutenção equipamentos confira seguir todos editais lançados especificidades universal edital 01/2014 apoia financeiramente projetos pesquisa científica tecnológica inovação diversas áreas conhecimento edital vai destinar todo r 23 milhões pesquisas aprovadas cada proposta deve ser r 50 mil propostas recebidas 31 março 2014. manutenção equipamentos edital 02/2014 financia reparos manutenção equipamentos laboratórios período garantia cujo custo aquisição sido superior r 100 mil todo distribuídos r 2 milhões propostas aprovadas propostas devem ser enviadas 2 abril programa pe

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 1),
    max_features=None,
    binary=False,
    use_idf=True
)
tfidf_matrix = vectorizer.fit_transform(df['doc'])

tfidf_matrix

<3080x18884 sparse matrix of type '<class 'numpy.float64'>'
	with 219300 stored elements in Compressed Sparse Row format>

In [9]:
pd.DataFrame(tfidf_matrix.todense(), columns=vectorizer.get_feature_names()).head()

,00,000,001,001087,002,002084,002086,003,003066,003360,...,último,últimos,única,únicas,único,únicos,úteis,útero,útil,útimo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer



tfidf_matrix


<3080x18884 sparse matrix of type '<class 'numpy.float64'>'
	with 219300 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcula todas as similaridades de cada linha
sim = cosine_similarity(tfidf_matrix)

In [44]:
vectorizer = TfidfVectorizer(
        analyzer='word',
        ngram_range=(1, 1),
        max_features=None,
        binary=False,
        use_idf=True
    )
data = df.append({'doc': "divulgado o Edital de iniciação científica"}, ignore_index=True)
tfidf_matrix = vectorizer.fit_transform(data['doc'])

from sklearn.metrics.pairwise import cosine_similarity

# Calcula todas as similaridades de cada linha
sim = cosine_similarity(tfidf_matrix)

sim.shape

l = []
for i in range(0, len(sim)-1):
    l.append((i, sim[i][-1]))
l = sorted(l, key=lambda x: x[1], reverse=True)[0:5]
l

[(2795, 0.4417390642181212),
 (7, 0.39012374489011686),
 (2376, 0.36097303709428197),
 (615, 0.3557144395873862),
 (612, 0.33367163337761924)]

In [48]:
data.iloc[2376].doc

'divulgado resultado processo seleção bolsistas iniciação científica ensino médio divulgado resultado processo seleção bolsistas iniciação científica ensino médio total 105 inscrições concorrer 13 bolsas iniciação científica pibic-jr cnpq referentes 11 projetos campus barbacena aprovados edital 09.2011. período vigência bolsas é março/2012 31/01/2013 veja resultado outras informações telefone 3693-8612'

In [45]:
data.iloc[-1].doc

'divulgado o Edital de iniciação científica'

In [56]:
def trazer_mais_proximas(noticia):    
    
    vectorizer = TfidfVectorizer(
        analyzer='word',
        ngram_range=(1, 1),
        max_features=None,
        binary=False,
        use_idf=True
    )
    data = df.append({'doc': noticia}, ignore_index=True)
    tfidf_matrix = vectorizer.fit_transform(data['doc'])

    from sklearn.metrics.pairwise import cosine_similarity

    # Calcula todas as similaridades de cada linha
    sim = cosine_similarity(tfidf_matrix)

    sim.shape

    l = []
    for i in range(0, len(sim)-1):
        l.append((i, sim[i][-1]))
    l = sorted(l, key=lambda x: x[1], reverse=True)[0:5]
    [print(data.iloc[x[0]].link) for x in l]
    return [data.iloc[x[0]].link for x in l]

In [57]:
trazer_mais_proximas("divulgado o Edital de iniciação científica")

http://www.barbacena.ifsudestemg.edu.br/destaques/divulgado-resultado-final-retificado-edital-042016-edital-projetos-iniciacao-cientifica
http://www.barbacena.ifsudestemg.edu.br/destaques/divulgado-resultado-provisorio-projetos-iniciacao-cientifica-encaminhados-ao-edital-052016
http://www.barbacena.ifsudestemg.edu.br/destaques/divulgado-resultado-processo-selecao-bolsistas-iniciacao-cientifica-ensino-medio-0
http://www.barbacena.ifsudestemg.edu.br/destaques/divulgado-resultado-provisorio-editais-iniciacao-cientifica-3-4-52013
http://www.barbacena.ifsudestemg.edu.br/destaques/publicada-retificacao-resultado-final-campus-barbacena-edital-selecao-projetos-iniciacao-c


['http://www.barbacena.ifsudestemg.edu.br/destaques/divulgado-resultado-final-retificado-edital-042016-edital-projetos-iniciacao-cientifica',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/divulgado-resultado-provisorio-projetos-iniciacao-cientifica-encaminhados-ao-edital-052016',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/divulgado-resultado-processo-selecao-bolsistas-iniciacao-cientifica-ensino-medio-0',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/divulgado-resultado-provisorio-editais-iniciacao-cientifica-3-4-52013',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/publicada-retificacao-resultado-final-campus-barbacena-edital-selecao-projetos-iniciacao-c']

In [58]:
trazer_mais_proximas('''Servidores e alunos do NEABI - Núcleo de Estudos Afro-Brasileiro e Indígena do Campus Barbacena participaram no dia 06 de novembro de 2018, do III ERAS - Encontro de Relações Raciais e Sociedade, no Campus João del-Rei.

Representado pelos servidores Adriano José Boratto e Vilma Maria Azevedo, foram ofertadas atividades no campo étnico-racial: Oficina de Capoeira e Oficina de Arte-Educação na Valorização da Cultura Africana; apresentação cultural: Dança Cênica Afro-brasileira; Banda Pitanga: Música Afro-brasileira Memorial a Clementina de Jesus; Apresentação Cultural de Capoeira e o Desafio de Hip-Hop.

Os estudantes participantes deste evento são integrantes dos projetos de extensão de abordagem étnico-racial, coordenado pelos membros do NEABI e pela servidora Leila Maria Pereira. Também participaram e integraram a equipe dezesseis estudantes das escolas públicas do Município, a Instituição São Miguel Arcanjo e da Escola Quilombola do Ponto Chique do Martelo; a diretora e uma professora da Escola Quilombola e quatro colaboradores externos voluntários dos projetos de extensão em questão, em especial Luciana de Oliveira (teatróloga); Flávia Sol (psicopedagoga); Talita S. de Paula da Silva (mestranda na UFSJ em Artes, Urbanidade e Sustentabilidade e Pedro Machado Caetano (músico).''')

http://www.barbacena.ifsudestemg.edu.br/destaques/servidores-alunos-neabi-nucleo-estudos-afro-brasileiro-indigena-campus-barbacena-participa
http://www.barbacena.ifsudestemg.edu.br/destaques/alunos-campus-barbacena-participam-projeto-%E2%80%9Cavaliacao-cana-acucar-hidrolisada-equinos%E2%80%9D
http://www.barbacena.ifsudestemg.edu.br/destaques/alunos-servidores-campus-barbacena-apresentam-trabalho-vlll-semana-pesquisa-extensao-lll-s
http://www.barbacena.ifsudestemg.edu.br/destaques/projeto-if-sudeste-mg-campus-barbacena-atende-comunidade-ponto-chique-martelo-seu-entorno
http://www.barbacena.ifsudestemg.edu.br/destaques/professora-curso-tecnico-agroindustria-integrado-ao-ensino-medio-fala-armazenamento-alimen


['http://www.barbacena.ifsudestemg.edu.br/destaques/servidores-alunos-neabi-nucleo-estudos-afro-brasileiro-indigena-campus-barbacena-participa',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/alunos-campus-barbacena-participam-projeto-%E2%80%9Cavaliacao-cana-acucar-hidrolisada-equinos%E2%80%9D',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/alunos-servidores-campus-barbacena-apresentam-trabalho-vlll-semana-pesquisa-extensao-lll-s',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/projeto-if-sudeste-mg-campus-barbacena-atende-comunidade-ponto-chique-martelo-seu-entorno',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/professora-curso-tecnico-agroindustria-integrado-ao-ensino-medio-fala-armazenamento-alimen']

In [ ]:
df.iloc[577].doc